#### Import

In [ ]:
import easyocr
import os
import torch
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import random
import glob
from tqdm import tqdm
import numpy as np
import time
from mmocr_eval import eval_ocr_metric
import re
import cv2 as cv
from icecream import ic

In [ ]:
# https://www.kaggle.com/code/fulrose/how-to-apply-new-font-to-matplotlib-easily/notebook
def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'

    # font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    # unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    # os.system(font_download_cmd)
    # os.system(unzip_cmd)


    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    try:
        # print(font_files)
        font_name = fm.FontProperties(fname=font_files[0]).get_name()
    except:
        print('Font not found')
        font_name = 'Kanit'
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

In [ ]:
font_download_url = 'https://fonts.google.com/download?family=Kanit'
change_matplotlib_font(font_download_url)

#### Install packges

In [ ]:
# !pip install matplotlib
# !pip install rapidfuzz

In [ ]:
torch.cuda.is_available()

#### Training

Load model by easyOCR API (https://www.jaided.ai/easyocr/documentation/)

In [ ]:
start = time.time()
model = 'crnn_100k_lr_25e-2' #folder name of model
iteration = 'iter_30000' # model .pt file name

model_path = os.path.join('trainer', 'saved_models',model)
# reader = easyocr.Reader(lang_list = ['en','th'],
#                     model_storage_directory = model_path,
#                     user_network_directory = os.path.join('my_model','user_network'),
#                     recog_network  = iteration,
#                     config_path = 'crnn', #configuration file name use to get character_list only  (so able to use for every model)
#                     gpu = True)
reader = easyocr.Reader(lang_list = ['en','th'],gpu = True )
end = time.time()
print(f'loading model time: {end - start}')

visualise testing with some image

In [ ]:
list_im = [789, 782, 575, 779, 589, 714, 999 ,683]
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))

font_name = 'Kanit'
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
all_pred = []
columns = 4
rows = 4
fig = plt.figure(figsize=(15, 15))
ax = []
for i in tqdm(range(columns*rows)):
    # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
    # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
    #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

    random_file_path = random.sample(filepaths,1)[0]
    # random_file_path = filepaths[1]
    # random_file_path = filepaths[list_im.pop(0)]
    img = plt.imread(random_file_path)
    result = reader.recognize(random_file_path)
    all_pred.append(result[0][1])
    ax.append(fig.add_subplot(rows, columns, i+1) )
    boxs = []
    txt = []
    for idx,i in enumerate(result):
        boxs.append(i[0])
        txt.append(i[1])
    plt.axis('off')
    if len(result) > 0:
        ax[-1].set_title(f'{idx+1} predict: {txt}' , fontsize=20)
        # for i in boxs:
        #     pts = np.array(i, np.int32)
        #     pts = pts.reshape((-1,1,2))
        #     cv.polylines(img,[pts],True,(0,0,0))

        
    else:
        ax[-1].set_title('No result', fontsize=20)
    plt.imshow(img)
fig.tight_layout() 
plt.show()
print(all_pred)

In [ ]:
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
label_path = 'trainer/all_data/testing/testing/label.txt'
with open(label_path, encoding='utf8') as file:
    label = np.loadtxt(file,dtype=str)
word_correct = 0

In [ ]:
# columns = 5
# rows = 200
# fig, ax = plt.subplots(rows, columns, figsize=(20, 20))
# ax.set_facecolor("white")
# counter = 0
# for r in tqdm(range(rows)):
#     for c in range(columns):
#         file_name = re.split(r'[/\\]',filepaths[counter])[-1]
#         file_path = os.path.join('trainer/all_data/testing/testing',file_name)
#         img = plt.imread(file_path)
#         ax[r, c].axis('off')
#         result = reader.readtext(file_path) 
#         if len(result) > 0:
#             txt = ''
#             for i in result:
#                 txt += (' '+i[1])
#             ax[r, c].set_title(f'{file_name} predict: {txt}')
#         else:
#             ax[r, c].set_title('No result')
#         ax[r, c].imshow(img)
#         counter += 1
# plt.subplots_adjust(top = 20)

testing with all files in dataset

In [ ]:
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# # filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))[:1000]
# filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
# label_path = 'trainer/all_data/testing/testing/label.txt'
# with open(label_path, encoding='utf8') as file:
#     label = np.loadtxt(file,dtype=str)

# start = time.time()
# # filepaths = filepaths[:3]
# pred = []
# gt = [] # ground truth
# name_img = []
# for i in tqdm(range(len(filepaths))):
#     label_img = label[i][1]
#     name_img.append(label[i][0])
#     result = reader.recognize(filepaths[i],detail=0)
#     try:
#         pred.append(result)
#     except:
#         pred.append('')
#     gt.append(label_img)
# end = time.time()
# print(f'time: {end - start}')

In [ ]:
# tmp = name_img.copy()
# output, false_list = eval_ocr_metric(pred, gt, tmp)
# # output, false_list = eval_ocr_metric(pred, gt, name_img)
# for k,v in output.items():
#     print(f'{k}: {v}')

visualise testing with flase prediction

In [ ]:

# plt.rcParams['xtick.labelsize'] = 20.0
# plt.rcParams['ytick.labelsize'] = 20.0
# columns = 3
# rows = 5
# fig = plt.figure(figsize=(15, 15))
# ax = []
# all_pred = []
# for i in tqdm(range(columns*rows)):
#     # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

#     random_file_name = random.sample(false_list,1)[0]
#     random_file_name = random_file_name[0]
#     random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)

#     img = plt.imread(random_file_path)
#     result = reader.recognize(random_file_path)
#     ax.append(fig.add_subplot(rows, columns, i+1) )
#     # boxs = []
#     txt = []
#     for idx,i in enumerate(result):
#         # boxs.append(i[0])
#         txt.append(i[1])
#     all_pred.append(txt)
#     plt.axis('off')
#     if len(result) > 0:
#         ax[-1].set_title(f'file: {random_file_name} {idx}: {txt} ', fontsize=20) 
#         # for i in boxs:
#         #         pts = np.array(i, np.int32)
#         #         pts = pts.reshape((-1,1,2))
#         #         cv.polylines(img,[pts],True,(0,0,0))
#     else:
#         ax[-1].set_title(f'file: {random_file_name} No result', fontsize=20)
#     plt.imshow(img)
# fig.tight_layout() 
# plt.show()
# print(all_pred)

testing with LMWN

In [ ]:

# plt.rcParams['xtick.labelsize'] = 20.0
# plt.rcParams['ytick.labelsize'] = 20.0
# columns = 2
# rows = 10
# fig = plt.figure(figsize=(20, 20))
# ax = []
# for i in range(columns*rows):
#     random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/LMWN')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_name = os.listdir(os.path.join('trainer/all_data/testing/LMWN'))[2]
#     random_file_path = os.path.join('trainer/all_data/testing/LMWN',random_file_name)
#     img = plt.imread(random_file_path)
#     result = reader.recognize(random_file_path) 
#     ax.append(fig.add_subplot(rows, columns, i+1))
#     plt.axis('off')
#     if len(result) > 0:
#         ax[-1].set_title(f'predict: "{result[0][1]}" ', fontsize=10)
#         # pts = np.array(result[0][0], np.int32)
#         # pts = pts.reshape((-1,1,2))
#         # cv.polylines(img,[pts],True,(255,0,0))
#     else:
#         ax[-1].set_title('No result',fontsize=10)
#     plt.imshow(img)
# fig.tight_layout() 
# plt.show()

testing with google ocr testset

In [ ]:
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/google_ocr_test_images','*.jpg'))
# # filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))[:1000]
# filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
# label_path = 'trainer/all_data/testing/google_ocr_test_images/image.txt'
# label= []
# with open(label_path, encoding='utf8') as file:
#     lines = file.readlines()
#     for line in lines:
#         label.append(eval(line))
# start = time.time()
# # filepaths = filepaths[:3]
# pred = []
# gt = [] # ground truth
# name_img = []
# for i in tqdm(range(len(filepaths))):
#     label_img = label[i]['text']
#     name_img.append(label[i]['filename'])
#     result = reader.recognize(filepaths[i],detail=0)
#     try:
#         pred.append(result)
#     except:
#         pred.append('')
#     gt.append(label_img)
# end = time.time()
# print(f'time: {end - start}')

In [ ]:
# tmp = name_img.copy()
# output, false_list = eval_ocr_metric(pred, gt, tmp)
# # output, false_list = eval_ocr_metric(pred, gt, name_img)
# for k,v in output.items():
#     print(f'{k}: {v}')

In [ ]:

# plt.rcParams['xtick.labelsize'] = 20.0
# plt.rcParams['ytick.labelsize'] = 20.0
# columns = 3
# rows = 5
# fig = plt.figure(figsize=(15, 15))
# ax = []
# all_pred = []
# for i in tqdm(range(columns*rows)):
#     # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

#     random_file_name = random.sample(false_list,1)[0]
#     random_file_name = random_file_name[0]
#     random_file_path = os.path.join('trainer/all_data/testing/google_ocr_test_images',random_file_name)

#     img = plt.imread(random_file_path)
#     result = reader.recognize(random_file_path)
#     ax.append(fig.add_subplot(rows, columns, i+1) )
#     # boxs = []
#     txt = []
#     for idx,i in enumerate(result):
#         # boxs.append(i[0])
#         txt.append(i[1])
#     all_pred.append(txt)
#     plt.axis('off')
#     if len(result) > 0:
#         ax[-1].set_title(f'file: {random_file_name} {idx}: {txt} ', fontsize=15) 
#         # for i in boxs:
#         #         pts = np.array(i, np.int32)
#         #         pts = pts.reshape((-1,1,2))
#         #         cv.polylines(img,[pts],True,(0,0,0))
#     else:
#         ax[-1].set_title(f'file: {random_file_name} No result', fontsize=15)
#     plt.imshow(img)
# fig.tight_layout() 
# plt.show()
# print(all_pred)

In [ ]:
def load_model(model_path, iteration):
    reader = easyocr.Reader(lang_list = ['en','th'],
                    model_storage_directory = model_path,
                    user_network_directory = os.path.join('my_model','user_network'),
                    recog_network  = iteration,
                    config_path = 'crnn', #configuration file name use to get character_list only  (so able to use for every model)
                    gpu = True)
    return reader

In [ ]:
# def test_all_testset(reader):
#     filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
#     filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
#     label_path = 'trainer/all_data/testing/testing/label.txt'
#     with open(label_path, encoding='utf8') as file:
#         label = np.loadtxt(file,dtype=str)

#     start = time.time()
#     # filepaths = filepaths[:3]
#     pred = []
#     gt = [] # ground truth
#     name_img = []
#     for i in tqdm(range(len(filepaths))):
#         label_img = label[i][1]
#         name_img.append(label[i][0])
#         result = reader.recognize(filepaths[i],detail=0)
#         try:
#             pred.append(result)
#         except:
#             pred.append('')
#         gt.append(label_img)
#     end = time.time()
#     print(f'time: {end - start}')
#     return pred, gt, name_img

loop testing

In [ ]:

# from IPython.display import clear_output
# model = 'crnn_100k_lr_25e-2' #folder name of model
# model_path = os.path.join('trainer', 'saved_models',model)

# files_list = os.listdir(model_path)

# result = {}
# for file in files_list:
#     if file.startswith('iter'):
#         iteration = file.split('.pth')[0]
#         reader = load_model(model_path, iteration)
#         pred, gt, name_img = test_all_testset(reader)
#         tmp = name_img.copy()
#         output, false_list = eval_ocr_metric(pred, gt, tmp)
#         # output, false_list = eval_ocr_metric(pred, gt, name_img)
#         result[iteration] = output
#         clear_output(wait=True)
# ic(result)

In [ ]:
# for k,v in result.items():
#     print(f'{k}: {v["word_acc"]}')

In [ ]:
txt = {'iter_18000': {'char_recall': 0.9089,
  'char_precision': 0.9404,
  'word_acc': 0.694,
  'word_acc_ignore_case': 0.705,
  'word_acc_ignore_case_symbol': 0.819,
  '1-N.E.D': 0.8929},
 'iter_30000': {'char_recall': 0.9001,
  'char_precision': 0.9572,
  'word_acc': 0.778,
  'word_acc_ignore_case': 0.786,
  'word_acc_ignore_case_symbol': 0.875,
  '1-N.E.D': 0.9208},
 'iter_20000': {'char_recall': 0.8954,
  'char_precision': 0.9658,
  'word_acc': 0.74,
  'word_acc_ignore_case': 0.749,
  'word_acc_ignore_case_symbol': 0.874,
  '1-N.E.D': 0.9182},
 'iter_26000': {'char_recall': 0.9163,
  'char_precision': 0.9624,
  'word_acc': 0.771,
  'word_acc_ignore_case': 0.784,
  'word_acc_ignore_case_symbol': 0.882,
  '1-N.E.D': 0.9331},
 'iter_10000': {'char_recall': 0.879,
  'char_precision': 0.945,
  'word_acc': 0.645,
  'word_acc_ignore_case': 0.667,
  'word_acc_ignore_case_symbol': 0.84,
  '1-N.E.D': 0.9091},
 'iter_28000': {'char_recall': 0.8989,
  'char_precision': 0.9727,
  'word_acc': 0.778,
  'word_acc_ignore_case': 0.787,
  'word_acc_ignore_case_symbol': 0.889,
  '1-N.E.D': 0.928},
 'iter_32000': {'char_recall': 0.9177,
  'char_precision': 0.9612,
  'word_acc': 0.813,
  'word_acc_ignore_case': 0.824,
  'word_acc_ignore_case_symbol': 0.896,
  '1-N.E.D': 0.9336},
 'iter_16000': {'char_recall': 0.9028,
  'char_precision': 0.9621,
  'word_acc': 0.772,
  'word_acc_ignore_case': 0.778,
  'word_acc_ignore_case_symbol': 0.882,
  '1-N.E.D': 0.932},
 'iter_4000': {'char_recall': 0.7782,
  'char_precision': 0.9182,
  'word_acc': 0.404,
  'word_acc_ignore_case': 0.426,
  'word_acc_ignore_case_symbol': 0.683,
  '1-N.E.D': 0.8424},
 'iter_22000': {'char_recall': 0.9031,
  'char_precision': 0.9538,
  'word_acc': 0.756,
  'word_acc_ignore_case': 0.765,
  'word_acc_ignore_case_symbol': 0.86,
  '1-N.E.D': 0.9113},
 'iter_8000': {'char_recall': 0.8546,
  'char_precision': 0.9629,
  'word_acc': 0.622,
  'word_acc_ignore_case': 0.642,
  'word_acc_ignore_case_symbol': 0.826,
  '1-N.E.D': 0.92},
 'iter_14000': {'char_recall': 0.8937,
  'char_precision': 0.9509,
  'word_acc': 0.704,
  'word_acc_ignore_case': 0.719,
  'word_acc_ignore_case_symbol': 0.86,
  '1-N.E.D': 0.9207},
 'iter_12000': {'char_recall': 0.8643,
  'char_precision': 0.9614,
  'word_acc': 0.641,
  'word_acc_ignore_case': 0.676,
  'word_acc_ignore_case_symbol': 0.853,
  '1-N.E.D': 0.9151},
 'iter_24000': {'char_recall': 0.9033,
  'char_precision': 0.9784,
  'word_acc': 0.812,
  'word_acc_ignore_case': 0.817,
  'word_acc_ignore_case_symbol': 0.905,
  '1-N.E.D': 0.945},
 'iter_6000': {'char_recall': 0.8499,
  'char_precision': 0.9326,
  'word_acc': 0.526,
  'word_acc_ignore_case': 0.552,
  'word_acc_ignore_case_symbol': 0.781,
  '1-N.E.D': 0.8894},
 'iter_2000': {'char_recall': 0.5429,
  'char_precision': 0.8213,
  'word_acc': 0.08,
  'word_acc_ignore_case': 0.09,
  'word_acc_ignore_case_symbol': 0.484,
  '1-N.E.D': 0.7085}}

In [ ]:
import matplotlib.pyplot as plt
x = []
y = []
key_sorted = sorted(txt, key= lambda x: int(x.split('_')[1]))
for k in key_sorted:
    x.append(int(k.split('_')[1]))
    y.append(txt[k]['word_acc'])
plt.figure(figsize=(8, 6), dpi=80)
plt.plot(x,y)
plt.title('Word Accuracy vs Iteration')
plt.xlabel('Iteration')
plt.ylabel('Word Accuracy')
plt.show()

In [ ]:
txt = {'iter_10000': {'1-N.E.D': 0.9003,
                            'char_precision': 0.942,
                            'char_recall': 0.844,
                            'word_acc': 0.513,
                            'word_acc_ignore_case': 0.547,
                            'word_acc_ignore_case_symbol': 0.817},
             'iter_12000': {'1-N.E.D': 0.9013,
                            'char_precision': 0.9471,
                            'char_recall': 0.8622,
                            'word_acc': 0.6,
                            'word_acc_ignore_case': 0.623,
                            'word_acc_ignore_case_symbol': 0.838},
             'iter_14000': {'1-N.E.D': 0.9238,
                            'char_precision': 0.9668,
                            'char_recall': 0.8563,
                            'word_acc': 0.605,
                            'word_acc_ignore_case': 0.637,
                            'word_acc_ignore_case_symbol': 0.865},
             'iter_16000': {'1-N.E.D': 0.9231,
                            'char_precision': 0.9547,
                            'char_recall': 0.8725,
                            'word_acc': 0.629,
                            'word_acc_ignore_case': 0.646,
                            'word_acc_ignore_case_symbol': 0.864},
             'iter_18000': {'1-N.E.D': 0.9142,
                            'char_precision': 0.9562,
                            'char_recall': 0.8666,
                            'word_acc': 0.686,
                            'word_acc_ignore_case': 0.697,
                            'word_acc_ignore_case_symbol': 0.864},
             'iter_2000': {'1-N.E.D': 0.4499,
                           'char_precision': 0.3657,
                           'char_recall': 0.0676,
                           'word_acc': 0.0,
                           'word_acc_ignore_case': 0.0,
                           'word_acc_ignore_case_symbol': 0.413},
             'iter_20000': {'1-N.E.D': 0.9064,
                            'char_precision': 0.9567,
                            'char_recall': 0.869,
                            'word_acc': 0.673,
                            'word_acc_ignore_case': 0.688,
                            'word_acc_ignore_case_symbol': 0.855},
             'iter_22000': {'1-N.E.D': 0.9378,
                            'char_precision': 0.9747,
                            'char_recall': 0.8731,
                            'word_acc': 0.715,
                            'word_acc_ignore_case': 0.728,
                            'word_acc_ignore_case_symbol': 0.896},
             'iter_24000': {'1-N.E.D': 0.9069,
                            'char_precision': 0.9573,
                            'char_recall': 0.8631,
                            'word_acc': 0.708,
                            'word_acc_ignore_case': 0.724,
                            'word_acc_ignore_case_symbol': 0.857},
             'iter_26000': {'1-N.E.D': 0.9403,
                            'char_precision': 0.98,
                            'char_recall': 0.8778,
                            'word_acc': 0.707,
                            'word_acc_ignore_case': 0.717,
                            'word_acc_ignore_case_symbol': 0.902},
             'iter_28000': {'1-N.E.D': 0.9286,
                            'char_precision': 0.9616,
                            'char_recall': 0.8907,
                            'word_acc': 0.706,
                            'word_acc_ignore_case': 0.72,
                            'word_acc_ignore_case_symbol': 0.889},
             'iter_30000': {'1-N.E.D': 0.9091,
                            'char_precision': 0.9588,
                            'char_recall': 0.8749,
                            'word_acc': 0.737,
                            'word_acc_ignore_case': 0.745,
                            'word_acc_ignore_case_symbol': 0.877},
             'iter_32000': {'1-N.E.D': 0.9235,
                            'char_precision': 0.9592,
                            'char_recall': 0.8898,
                            'word_acc': 0.736,
                            'word_acc_ignore_case': 0.744,
                            'word_acc_ignore_case_symbol': 0.885},
             'iter_34000': {'1-N.E.D': 0.9431,
                            'char_precision': 0.9801,
                            'char_recall': 0.8828,
                            'word_acc': 0.765,
                            'word_acc_ignore_case': 0.772,
                            'word_acc_ignore_case_symbol': 0.906},
             'iter_4000': {'1-N.E.D': 0.6686,
                           'char_precision': 0.73,
                           'char_recall': 0.5911,
                           'word_acc': 0.124,
                           'word_acc_ignore_case': 0.133,
                           'word_acc_ignore_case_symbol': 0.473},
             'iter_6000': {'1-N.E.D': 0.848,
                           'char_precision': 0.8945,
                           'char_recall': 0.7397,
                           'word_acc': 0.337,
                           'word_acc_ignore_case': 0.353,
                           'word_acc_ignore_case_symbol': 0.717},
             'iter_8000': {'1-N.E.D': 0.8632,
                           'char_precision': 0.9066,
                           'char_recall': 0.8096,
                           'word_acc': 0.435,
                           'word_acc_ignore_case': 0.465,
                           'word_acc_ignore_case_symbol': 0.754}}

In [ ]:
import matplotlib.pyplot as plt
x = []
y = []
key_sorted = sorted(txt, key= lambda x: int(x.split('_')[1]))
for k in key_sorted:
    x.append(int(k.split('_')[1]))
    y.append(txt[k]['word_acc'])
plt.figure(figsize=(8, 6), dpi=80)
plt.plot(x,y)
plt.title('Word Accuracy vs Iteration')
plt.xlabel('Iteration')
plt.ylabel('Word Accuracy')
plt.show()